In [12]:
import csv
import numpy as np
from model import StandardScaler, LogisticRegressionScratch
from train import load_raisin_csv, train_val_split, accuracy
import json

# Train

In [13]:
X, y, feature_names, mapping = load_raisin_csv("Raisin_Dataset.csv", label_col="Class")
X_tr, y_tr, X_va, y_va = train_val_split(X, y, val_frac=0.2, seed=42)

scaler = StandardScaler()
X_tr_s = scaler.fit_transform(X_tr)
X_va_s = scaler.transform(X_va)

model = LogisticRegressionScratch(lr=0.1, n_iters=2000).fit(X_tr_s, y_tr)

tr_acc = accuracy(y_tr, model.predict(X_tr_s))
va_acc = accuracy(y_va, model.predict(X_va_s))
tr_loss = model.loss(X_tr_s, y_tr)
va_loss = model.loss(X_va_s, y_va)

print("Label mapping:", mapping)
print(f"Train loss={tr_loss:.4f} acc={tr_acc:.4f}")
print(f"Val   loss={va_loss:.4f} acc={va_acc:.4f}")

model.save_npz("model.npz", scaler, feature_names)
print("Saved -> model.npz")

Label mapping: {'Besni': 0, 'Kecimen': 1}
Train loss=0.3554 acc=0.8722
Val   loss=0.3494 acc=0.8444
Saved -> model.npz


# Predict

In [15]:
model, scaler, feature_names = LogisticRegressionScratch.load_npz("model.npz")

# Example input: {"Area": 875.0, "Perimeter": 110.1, ...}
raw = json.loads(input().strip())

x = np.array([float(raw[name]) for name in feature_names], dtype=np.float64)[None, :]
x_s = scaler.transform(x)

p1 = float(model.predict_proba(x_s)[0])
pred = int(p1 >= 0.5)

print(json.dumps({"p_class1": p1, "pred": pred}))

KeyError: '\ufeffArea'

In [2]:
from app.model import *